In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import os
import pinecone
from langchain.vectorstores import Pinecone
from omegaconf import OmegaConf

load_dotenv(find_dotenv())

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  
    environment=os.getenv('PINECONE_ENV')  
)

/home/brayps/.cache/pypoetry/virtualenvs/table-extract-2ivw7RXS-py3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [30]:
# Use OpenAI for the browser
template = """
Look up this paper {doi}
Provide the following information ({columns}) from the paper. Leave the cell blank if the result is unclear.
""" # ! this one doesn't work?
template = """
Task: 
Write a comma-separated list of {columns} from the following paper. Leave the cell blank if the result is unclear.
Paper: {doi}
"""
template = """
Paper: {doi}
Provide the following information ({columns}) from the paper. Leave the cell blank if the result is unclear.
"""
template = """
Paper: {doi}
Provide the following information ({columns}) from the paper.
Information in each column must be under 50 words.
Leave the cell blank if the result is unclear.
Format the output as a markdown table
Output:
"""
prompt = PromptTemplate(
    input_variables=["columns", "doi"],
    template=template,
)
query = OmegaConf.load("query.yaml")
print(query)
prompt.format(**query)

{'doi': 1912.07383, 'columns': 'year published, research objective, key contribution'}


'\nPaper: 1912.07383\nProvide the following information (year published, research objective, key contribution) from the paper.\nInformation in each column must be under 50 words.\nLeave the cell blank if the result is unclear.\nFormat the output as a markdown table\nOutput:\n'

### Returns
>2019, Investigate the relationship between online ratings and revenue in the restaurant industry, Establish a dynamic linear model for prediction and forecasting of online ratings and restaurant revenue.'


- uhh that didn't work

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)
tools = load_tools(
    ["arxiv"],
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [32]:
res = agent_chain.run(
    prompt.format(**query)
)



> Entering new AgentExecutor chain...
I should use the arxiv tool to search for the paper and extract the required information.
Action: arxiv
Action Input: 1912.07383
Observation: Published: 2019-12-12
Title: A Survey of Predictive Maintenance: Systems, Purposes and Approaches
Authors: Yongyi Ran, Xin Zhou, Pengfeng Lin, Yonggang Wen, Ruilong Deng
Summary: This paper provides a comprehensive literature review on Predictive
Maintenance (PdM) with emphasis on system architectures, purposes and
approaches. In industry, any outages and unplanned downtime of machines or
systems would degrade or interrupt a company's core business, potentially
resulting in significant penalties and unmeasurable reputation loss. Existing
traditional maintenance approaches suffer from some assumptions and limits,
such as high prevent/repair costs, inadequate or inaccurate mathematical
degradation processes and manual feature extraction. With the trend of smart
manufacturing and development of Internet of Thi

In [33]:
print(res)

| Year Published | Research Objective | Key Contribution |
|----------------|--------------------|------------------|
| 2019           | Predictive Maintenance | A comprehensive literature review on Predictive Maintenance (PdM) with emphasis on system architectures, purposes, and approaches. Provides a high-level view of PdM system architectures, maintenance purposes/objectives, and a review of fault diagnosis and prognosis approaches in PdM systems. Introduces future research directions. |
